In [72]:
import pandas as pd
import glob
import os

INPUT_FOLDER = "bom_inputs"   # ide rakd a CSV fájlokat
OUTPUT_FILE = "MASTER_BOM_OUTPUT.xlsx"

In [94]:
def load_bom_csv(path):
    df = pd.read_csv(path, sep=';', header=0, index_col=False)
    df['Source'] = os.path.basename(path)
    return df

.csv fájlok

In [95]:
INPUT_FOLDER = "bom_inputs"

# Minden CSV fájl beolvasása
files = glob.glob(os.path.join(f"./{INPUT_FOLDER}/*.csv"))
if not files:
    print("Nincs CSV a bom_inputs mappában.")

.csv-k beolvasása

In [103]:
dfs = [load_bom_csv(f) for f in files]
# MASTER tábla
master = pd.concat(dfs)

In [104]:
master

,Id,Designator,Footprint,Quantity,Designation,Supplier and ref,Unnamed: 6,Source
0,1,"SW2,SW4,SW5,SW7,SW3,SW6,SW1",SW_Cherry_MX_1.00u_PCB,7,Cherry Switch Single,NaN,NaN,buttons.csv
1,2,J1,JST_PH_B9B-PH-K_1x09_P2.00mm_Vertical,1,Conn_01x09_Pin,NaN,NaN,buttons.csv
2,3,"R13,R2,R1,R8,R3,R7,R9",R_0603_1608Metric,7,1k,NaN,NaN,buttons.csv
3,4,"C2,C3,C6,C1,C4,C7,C5",C_0603_1608Metric,7,100n,NaN,NaN,buttons.csv
4,5,"R14,R12,R5,R6,R11,R4,R10",R_0603_1608Metric,7,10k,NaN,NaN,buttons.csv
...,...,...,...,...,...,...,...,...
11,12,R2,R_0603_1608Metric,1,10k,NaN,NaN,numpad.csv
12,13,R19,R_0603_1608Metric,1,100k,NaN,NaN,numpad.csv
13,14,R21,R_0603_1608Metric,1,4k7,NaN,NaN,numpad.csv
14,15,U1,SOIC-8_3.9x4.9mm_P1.27mm,1,LT1632,NaN,NaN,numpad.csv


In [105]:
# Normalizáljuk az oszlopneveket
master.columns = [c.strip().replace(" ", "_") for c in master.columns]

# Egy kulcs képzése (Designation + Footprint)
master["Key"] = (
    master["Designation"].astype(str).str.strip()
    + "_"
    + master["Footprint"].astype(str).str.strip()
)

# Összesített rendelési lista
summary = (
    master.groupby("Key")["Quantity"]
    .sum()
    .reset_index()
    .sort_values("Quantity", ascending=False)
)

with pd.ExcelWriter(OUTPUT_FILE) as writer:
        master.to_excel(writer, sheet_name="MASTER", index=False)
        summary.to_excel(writer, sheet_name="ORDER_SUMMARY", index=False)

In [107]:
summary

,Key,Quantity
54,TestPoint_Small_PinSocket_1x01_P2.54mm_Vertical,280
9,1k_R_0603_1608Metric,36
27,Cherry Switch Single_SW_Cherry_MX_1.00u_PCB,17
4,10k_R_0603_1608Metric,14
3,100n_C_0603_1608Metric,13
2,100k_R_0603_1608Metric,12
55,TestPoint_TestPoint_Pad_D2.0mm,9
38,GREEN_LED_D_0603_1608Metric,7
1,100R_R_0603_1608Metric,7
24,BSS138_SOT-23,6
